# Dval - Shapley for data valuation

This notebook introduces Dval, a library to evaluate the importance of single datapoints in the performance of machine learning models.
We will go through the foundations of the library, its main entry-points and capabilities working with a real dataset of music tracks.
We will also highlight the advantages of using our library over vanilla data-shapley calculations, showing explicitly the advantages in runtime and efficiency for large datasets.

In [ ]:
%load_ext autoreload
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd


In [ ]:
data = pd.read_csv('../data/top_hits_spotify_dataset.csv')
data = data[data['year']> 2014]

In [ ]:
data.head()

In [ ]:
target_column = 'popularity'
y = data[target_column]
X = data.drop(target_column, axis=1)
X['genre'] = data['genre'].astype('category').cat.codes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [ ]:
song_name = X_train['song']
artist = X_train['artist']
X_train = X_train.drop(['song', 'artist'], axis=1)
X_test = X_test.drop(['song', 'artist'], axis=1)

Note: Make sure to restart (or simply start if it is not already running) your memcache. In the terminal, type

`sudo service memcached restart`

In [ ]:
from valuation.shapley import create_utility, shapley_dval
utility = create_utility(model=RandomForestRegressor(n_estimators=10), x_train=X_train, y_train=y_train, x_test=X_test, y_test=y_test, scoring='r2', data_groups=artist,)
values, val_std = shapley_dval(utility, max_iterations=100, num_jobs=20)

In [ ]:
dval_df = pd.DataFrame(list(zip(values.keys(), values.values(), val_std.values())), columns=['song', 'shapley_dval', 'dval_std'])
low_dval = dval_df.iloc[:30]
plt.figure(figsize=(20, 5))
plt.errorbar(x=low_dval['song'], y=low_dval['shapley_dval'], yerr=low_dval['dval_std'], fmt='o')
plt.xticks(rotation=45)

In [ ]:
values, val_std = shapley_dval(utility, max_iterations=100, num_jobs=20)

In [ ]:
dval_df = pd.DataFrame(list(zip(values.keys(), values.values(), val_std.values())), columns=['song', 'shapley_dval', 'dval_std'])
low_dval = dval_df.iloc[:30]
plt.figure(figsize=(20, 5))
plt.errorbar(x=low_dval['song'], y=low_dval['shapley_dval'], yerr=low_dval['dval_std'], fmt='o')
plt.xticks(rotation=45)

In [ ]:
high_dval = dval_df.iloc[-30:]
plt.figure(figsize=(20, 5))
plt.errorbar(x=high_dval['song'], y=high_dval['shapley_dval'], yerr=high_dval['dval_std'], fmt='o')
plt.xticks(rotation=45)

## Evaluation of anomalous data

In [ ]:
new_drake_popularity = data[data['artist'] == 'Drake']['popularity'] - data[data['artist'] == 'Drake']['popularity']
data.loc[data['artist'] == 'Drake', 'popularity'] = new_drake_popularity

In [ ]:
plt.figure(figsize=(9, 6))
plt.scatter(data.y_test, predictions)
plt.plot([0, 50], [0, 50], '--k')
plt.xlabel('True')
plt.ylabel('Predicted');

In [ ]:
%autoreload

In [ ]:
from valuation.shapley import permutation_exact_shapley, truncated_montecarlo_shapley
from valuation.utils import map_reduce, Utility
from valuation.reporting.scores import compute_fb_scores
from valuation.reporting.plots import shapley_results

# Naive Shapley

We want to examine how the shapley values change with train and test data. In particular, we want to examine how robust data shapley values are to out of distribution input data. To do so, we will progressively add more and more outliers to our train dataset

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from valuation.utils.dataset import polynomial_dataset, polynomial

d, coeffs = polynomial_dataset(np.random.randint(-3, 3, size=3))  
model = make_pipeline(PolynomialFeatures(len(coeffs)-1), LinearRegression())

n = len(d)
model.fit(d.x_train, d.y_train)
predicted = [model.predict(d.x_test)]

x_cont = d.x_train.reshape(-1,) + np.random.uniform(-0.05, 0.05, size=len(d))
x_cont = x_cont[::2]
y_cont = polynomial(np.random.normal(loc=coeffs, scale=0.3), x_cont)
xtrain = np.concatenate([d.x_train, x_cont.reshape(-1, 1)], axis=0)
ytrain = np.concatenate([d.y_train, y_cont.reshape(-1,)], axis=0)
for i in range(len(d), len(xtrain)):
    model.fit(xtrain[:i+1], ytrain[:i+1])
    ypred = model.predict(d.x_test)
    predicted.append(ypred)

test_indices = np.argsort(d.x_test, axis=0).reshape(-1, )
xx = np.arange(-1, 1, 0.1)
yy = polynomial(coeffs, xx)

from matplotlib import pyplot as plt
for i, ypred in enumerate(predicted):
    plt.figure(dpi=300)
    plt.scatter(d.x_train[:n], d.y_train[:n], label="Training (in-dist)")
    plt.scatter(d.x_test, d.y_test, label="Test")

    if i > 0:
        plt.scatter(x_cont[:i], y_cont[:i], label="Training (out of dist)")
    plt.plot(xx, yy, label="True")
    plt.plot(d.x_test[test_indices], ypred[test_indices], label="Predicted")
    plt.ylim(min(d.y_train[:n].min(), y_cont.min()) - 1,
                max(d.y_train[:n].max(), y_cont.max()) + 1)
    plt.legend()
    plt.title(d.description)
    plt.show()

d.x_train = xtrain
d.y_train = ytrain

from valuation.shapley import combinatorial_exact_shapley
from valuation.utils import Utility
u = Utility(model, d, "neg_median_absolute_error")
values = combinatorial_exact_shapley(u, progress=False)
high_to_low = list(reversed(values))

take = 5
plt.figure(dpi=300)
plt.scatter(d.x_train[:n], d.y_train[:n], label="Training (in-dist)")
plt.scatter(d.x_test, d.y_test, label="Test")
plt.scatter(d.x_train[high_to_low][:take], d.y_train[high_to_low][:take],
            marker='x', label='High value')
plt.scatter(x_cont, y_cont, label="Training (out of dist)")
plt.plot(xx, yy, label="True")
plt.plot(d.x_test[test_indices], predicted[-1][test_indices], label="Predicted")

model.fit(d.x_train[high_to_low][:take], d.y_train[high_to_low][:take])
ypred = model.predict(d.x_test)
plt.plot(d.x_test, ypred, label='High: prediction')
plt.title(d.description)

plt.legend()
plt.show()

# MCShapley

In [ ]:
max_iterations = 200
utility = Utility(
    model,
    data,
    scoring=None,
    enable_cache=False,
)
fun = partial(permutation_exact_shapley, utility=utility, progress=True)
values_nmcs, hist_nmcs = map_reduce(fun, num_runs=10, num_jobs=160)

In [ ]:
scores_nmcs = compute_fb_scores(model=model, data=data, values=values_nmcs)

In [ ]:
scores_nmcs.update({'max_iterations': max_iterations, 'score_name': "$R^2$"})
shapley_results(scores_nmcs)

# Truncated MC Shapley

In [ ]:
params = {'bootstrap_iterations': 200,
          'min_scores': 10,
          'score_tolerance': 0.1,
          'min_values': 10,
          'value_tolerance': 1e-2,
          'max_iterations': 0.5*len(data)}
fun = partial(truncated_montecarlo_shapley, 
              model, data, num_workers=160, worker_progress=False, **params)

In [ ]:
values_mcs, hist_mcs = map_reduce(fun, data=data, num_runs=10)

In [ ]:
scores_mcs = compute_fb_scores(model=model, data=data, values=values_mcs)

In [ ]:
scores_mcs.update({'max_iterations': params['max_iterations'], 'score_name': "$R^2$"})
shapley_results(scores_mcs)